<a href="https://colab.research.google.com/github/gabrielaugustavo/LLM/blob/main/GSI073_aula0_redes_neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [ ]:
import torch; import sklearn; from torch import nn;from sklearn.model_selection import train_test_split

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X = iris.data        # 4 features: sépalas e pétalas
y = (iris.target == 1).astype(float)  # 1 se Versicolor, 0 caso contrário

# 2. Preparar dados para pytorch
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, random_state=44, stratify=y
)

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float().view(-1, 1)

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float().view(-1, 1)


## Definição do modelo e treinamento

In [ ]:
# 3. Definir modelo
import torch.nn.functional as F
class RedeNeural(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RedeNeural, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.elu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Criar modelo
modelo = RedeNeural(4, 100, 1)  # 4 features → 1 saída (probabilidade de ser Versicolor)

import copy
modelo_clonado = copy.deepcopy(modelo)

learning_rate = 0.1

# Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()  # combinação de sigmoid + BCE
optimizer = torch.optim.SGD(modelo.parameters(), lr=learning_rate)

## Execução do treinamento com optimizer SGD

In [ ]:
# Loop de treino
for epoch in range(1000):
    optimizer.zero_grad()           # Limpa gradientes
    outputs = modelo(X_train)             # Forward
    loss = funcao_perda(outputs, y_train) # Calcula perda
    loss.backward()                 # Calcula derivadas do gradiente
    optimizer.step()                # Aplica regra de alteração dos parâmetros

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

Teste de acurácia

In [ ]:
with torch.no_grad():
    outputs_train = modelo(X_test)
    predicted_train = (torch.sigmoid(outputs_train) >= 0.5).float()
    accuracy_train = (predicted_train == y_test).float().mean() * 100

print(f"Acurácia no treino: {accuracy_train.item():.4f}")

# Treino com regra de gradiente descendente manual

In [ ]:
for epoch in range(1000):
    optimizer.zero_grad()           # Limpa gradientes
    outputs = modelo_clonado(X_train)     # Forward
    loss = funcao_perda(outputs, y_train) # Calcula perda
    loss.backward()                 # Calcula derivadas do gradiente

    with torch.no_grad():
        for param in modelo_clonado.parameters():
            param -= learning_rate * param.grad  # regra de atualização de pesos

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/1000], Loss: {loss.item():.4f}")

In [ ]:
with torch.no_grad():
    outputs_train = modelo_clonado(X_test)
    predicted_train = (torch.sigmoid(outputs_train) >= 0.5).float()
    accuracy_train = (predicted_train == y_test).float().mean() * 100

print(f"Acurácia no treino: {accuracy_train.item():.4f}")